# Asignement 3 = Stag Hunt model 
By Māra Učelniece 

- Response : there is not a specific data set we have to use, but just apply the evaluation of the structures to the EV's case that we are attempting to create policy interventions for. 

### Task : 
Evaluate how varying structures relate to policy and in which cases they might apply. 
   

In [1]:
# Imports 
    # Copied from exmaple notebook 
    
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx
import numpy as np
import pandas as pd
import csv
import random
from typing import Iterable, List, Dict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed


# Can import the functions from the course_matterial 
    # Example : from file import function
# from course_matterial.ev_core import *     
    # Example to import all functions
        # from ./course_matterial/ev_core.py import *
#print("Done")

C:\Users\Mara\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Mara\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


## 0. Implement a network agent based model

In [2]:
from new_core import *
#%run new_core.py
print("Done")

Done


## 1. Baseline System Analysis

In [ ]:
# Evaluate trough plots 
# Noah doing this part 

## 2. Network Structure Analysis

Seems that the follwoing will be done with plots 
- speed of adoption,
- probability of reaching the high-adoption equilibrium,
- cluster formation,
- network-specific sensitivity to tipping

In [4]:
# Mara doing this part   

# Mainly could use ev_plotting for this,
    # but there is also some matterial in ev_experiments

In [3]:
# Speed of adoption

    # Quantified in a number
def calculate_adoption_rate(X_traj: np.ndarray) -> float:
    initial_adoption = X_traj[0]
    final_adoption = X_traj[-1]
    total_steps = len(X_traj)
    return (final_adoption - initial_adoption) / total_steps
    
    # Time to treashold ~> high adoption
        # May not capture overall dynamics 
def time_to_reach_threshold(X_traj: np.ndarray, threshold: float) -> int:
    for t, X in enumerate(X_traj):
        if X >= threshold:
            return t
    return len(X_traj)  # Return total steps if threshold not reached

    # Time series analysis : easy to interpret 
def plot_time_series(X_traj: np.ndarray, I_traj: np.ndarray, out_dir: str, X0: float, I0: float):
    t = np.arange(len(X_traj))
    plt.figure(figsize=(6, 4))
    plt.plot(t, X_traj, label="X(t)")
    plt.plot(t, I_traj, label="I(t)")
    plt.xlabel("time step")
    plt.ylabel("value")
    plt.title(f"Time series X/I  X0={X0:.2f}, I0={I0:.2f}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"timeseries_X0_{int(X0*100)}_I0_{int(I0*100)}.png"))
    plt.close()


# Probabaility of reaching high-adoption equilibrium 
    # Finan adoption fractio, but no account for fluctuation 
def calculate_final_adoption(X_traj: np.ndarray) -> float:
    return X_traj[-1]
    
    # Stability analysis 
def is_high_adoption_equilibrium(X_traj: np.ndarray, threshold: float) -> bool:
    return np.mean(X_traj[-10:]) >= threshold  # Check last 10 steps for stability

    # Probalistic measure to reach high adoption
def monte_carlo_high_adoption_probability(trials: int, model_params: dict, threshold: float) -> float:
    successes = 0
    for _ in range(trials):
        final_adoption, _ = run_trial(**model_params)
        if final_adoption >= threshold:
            successes += 1
    return successes / trials



# Cluster formation 
    # Clustering coeficient analysis ~> tendency 
def calculate_clustering_coefficient(G: nx.Graph) -> float:
    return nx.average_clustering(G)

    # Spatial Adoption patterns  
def plot_spatial_adoption(G: nx.Graph, node_agent_map: Dict[int, EVAgent], out_dir: str):
    fig, ax = plt.subplots(figsize=(6,6))
    pos = nx.spring_layout(G, seed=42)
    color_map = ["green" if agent.strategy=="C" else "red" for agent in node_agent_map.values()]
    nx.draw(G, pos, node_color=color_map, with_labels=True, ax=ax)
    plt.savefig(os.path.join(out_dir, "spatial_adoption.png"))
    plt.close(fig)  

    
# Could add comunity detection, to find distinct ones, 
    # But mainly useful doe understanding localised adoption patterns 
    
        # No need to add : discussed 

In [6]:
# Network-Specific sesnitivity to tipping 

    # Sensitivity analysis 
        # qunatifying impact of parameter changes on adoption 
def sensitivity_analysis(network_type: str, param_range: Iterable[float], model_params: dict, out_dir: str):
    results = []
    for param in param_range:
        model_params['beta_I'] = param
        final_adoption, _ = run_trial(**model_params)
        results.append(final_adoption)
    plt.plot(param_range, results, label=network_type)
    plt.xlabel('Parameter Value')
    plt.ylabel('Final Adoption')
    plt.title(f'Sensitivity Analysis for {network_type}')
    plt.legend()
    plt.savefig(os.path.join(out_dir, f"sensitivity_{network_type}.png"))
    plt.close()
    

    # Topping point estimation 
        # identifying critical points 
def estimate_tipping_point(X0_vals: List[float], I0_vals: List[float], model_params: dict, threshold: float, out_dir: str):
    tipping_points = []
    for X0 in X0_vals:
        for I0 in I0_vals:
            model_params['X0_frac'] = X0
            model_params['I0'] = I0
            final_adoption, _ = run_trial(**model_params)
            if final_adoption >= threshold:
                tipping_points.append((X0, I0))
    plt.scatter(*zip(*tipping_points))
    plt.xlabel('Initial Adoption X0')
    plt.ylabel('Initial Infrastructure I0')
    plt.title('Tipping Points')
    plt.savefig(os.path.join(out_dir, "tipping_points.png"))
    plt.close()
    
    
    # Phase plot analysis 
        # visualisation between adoption and infustructutre 
def plot_phase_trajectory(X_traj: np.ndarray, I_traj: np.ndarray, out_dir: str, X0: float, I0: float):
    plt.figure(figsize=(5, 5))
    plt.plot(I_traj, X_traj, marker=".", linewidth=1)
    plt.xlabel("I(t)")
    plt.ylabel("X(t)")
    plt.title(f"Phase plot X vs I  X0={X0:.2f}, I0={I0:.2f}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"phase_X0_{int(X0*100)}_I0_{int(I0*100)}.png"))
    plt.close()

In [ ]:
# Final adoption 

In [21]:
# Adjustement of the main 
    # to include the senitivity analysis 

def main():
    out_dir = "output_second_part"
    os.makedirs(out_dir, exist_ok=True)

    # ---- Config ----
        # Payoff + dynamic parameters
    a0 = 2.0      # base payoff / benifit of cooperation
    b = 1.0       # defector payoff
    g_I = 0.05    # infastructure growth rate/ response 
    beta_I = 2.0  # network effect / additional benifit 
                        # / infastructure payoff 
    
        # Network parameters 
    network_type = "ER"  # "ER", "WS", or "BA"
    n_nodes = 200    # nr of agents 
    p = 0.05    # prob of edge existing / connection prob  
    m = 2       # nr of edges from node / for BA and WS ... 
    
    X0_frac = 0.1   # inital fraction of adopters 
    I0 = 0.5      # baseline benifit/initial infastructure  

        # Strategy update 
    strategy_choice_func = "imitate" # or "logit"
    # tau = 1.0        # temp for loglit 
    
        # Simulation parameters 
    T = 200     # nr of simulation steps 
    threshold = 0.9  # Adoption fraction
    trials = 30  # simulations for probabilities 

    

    model_params = {
        "X0_frac": X0_frac,
        "I0": I0,
        "a0": a0,
        "beta_I": beta_I,
        "b": b,
        "g_I": g_I,
        "T": T,
        "network_type": network_type,
        "n_nodes": n_nodes,
        "p": p,
        "m": m,
        "seed": 42,
        "strategy_choice_func": strategy_choice_func,
         # "tau": tau,  # sodt max randomness parameter 
        "record_trajectory": True, # ensures saving x and i over time 
    }

    # ---- Run a single trial ----
    final_adoption, (X_traj, I_traj) = run_trial(**model_params)

    # ---- Adoption metrics ----
    adoption_rate = calculate_adoption_rate(X_traj)
    time_to_thresh = time_to_reach_threshold(X_traj, threshold)
    final_adopt = calculate_final_adoption(X_traj)
    high_eq = is_high_adoption_equilibrium(X_traj, threshold)
    high_prob = monte_carlo_high_adoption_probability(trials, model_params, threshold)

    print(f"Final adoption: {final_adopt:.3f}")
    print(f"Adoption rate: {adoption_rate:.4f} per step")
    print(f"Time to reach threshold {threshold}: {time_to_thresh} steps")
    print(f"High adoption equilibrium: {high_eq}")
    print(f"Probability of reaching high adoption (Monte Carlo {trials} trials): {high_prob:.3f}")

    # ---- Network metrics ----
    G = EVStagHuntModel(
        initial_ev=int(round(X0_frac * n_nodes)),
        a0=a0,
        beta_I=beta_I,
        b=b,
        g_I=g_I,
        I0=I0,
        seed=42,
        network_type=network_type,
        n_nodes=n_nodes,
        p=p,
        m=m,
        strategy_choice_func=strategy_choice_func,
        #tau=tau,
    ).G

    clustering = calculate_clustering_coefficient(G)
    print(f"Average clustering coefficient: {clustering:.3f}")

    # ---- Phase plot ----
    plot_phase_trajectory(X_traj, I_traj, out_dir, X0_frac, I0)

    # ---- Spatial adoption ----
    node_agent_map = {i: EVAgent(i, None, "D") for i in G.nodes()}
    # Assign strategies from final adoption for plotting
    for i, agent in enumerate(node_agent_map.values()):
        agent.strategy = "C" if i < int(final_adoption * n_nodes) else "D"

    plot_spatial_adoption(G, node_agent_map, out_dir)

    print("Analysis completed. Plots and metrics saved to:", out_dir)

    
    plot_time_series(X_traj, I_traj, out_dir, X0_frac, I0)
    
if __name__ == "__main__":
    main()

Final adoption: 0.000
Adoption rate: -0.0003 per step
Time to reach threshold 0.9: 400 steps
High adoption equilibrium: False
Probability of reaching high adoption (Monte Carlo 30 trials): 0.000
Average clustering coefficient: 0.046
Analysis completed. Plots and metrics saved to: output_second_part


### Result evaluation 
After a few runs : 
- ER :
    - If the adoption rte is even sligtly negative, with even values of -0.0005, then the final adoption is 0 not 1. 
    - The average clustering coeficient is about 0.5 in with current adjusments inn  almost all of teh parameters .

- WS 
.... 

### Discussion with Noah

Just stick to the one value of clustering coeficient for cluster formaton evaluation, yes? 

For the time series plot adjsutemnts they will depend on the policy implementations that we are going to discuss 

        Infastructure, and time 

For the sensitivity check heatmap and the possible three implemented. 


#### Task further:  

For policy intervention => the network represents the area, but as a general depiction, as they do not fully represtent ... 
    
Mara = infastructure shocks 

    Do the comparison with local as high centrality vs global 
    
    Do global frisrt and then base local on the targetted seeding noah does . 
    
Noah = targeted seeding 


## 3. Policy Intervention Designand Evaluation

In [5]:
# For this one the ev_experiments mainly perevelent: 
    # policy_subsidy_factory = Create a policy that temporarily boosts coordination payoffs
    # policy_infrastructure_boost_factory = Create a policy that injects infrastructure at a specific step
    
# However, these are only two interventions, and we wnat to test possibly more or others 
    # Can still be used for insparation / a basis to start from 
    